In [1]:
from leabra7 import program as prg
from leabra7 import net
from leabra7 import specs
%matplotlib inline

# leabra7 new interface 

Noam and I have done lots of good stuff lately (netin scaling, average kwta, fixing bugs, one-to-one projns, learning averages, etc.).
This is the main push I've been doing though.

## First, a demo

In [ ]:
# Declare the network structure
network = net.Net()
network.new_layer("input", size=3)
network.new_layer("output", size=3, spec=specs.LayerSpec(
    log_on_cycle=["unit_act", "unit_spike"]))
network.new_projn("projn1", pre="input", post="output")

# Declare a simple program
program = prg.Program([
        prg.HardClamp("input", [0.7]),  # Note the richness of the messages
        prg.Loop([
            prg.Cycle()
        ], num_iter=50)
])

# Run the program
network.execute(program)

# Slice, dice, and plot the logs
layer_logs, unit_logs = network.logs("cycle", "output")
unit_logs.loc[unit_logs["unit"] == 0][["act", "spike", "time"]].plot(x="time")

## Motivation

I was trying to implement **XCAL**. But, how do we define plus and minus phases without spaghetti code? We want to easily modify program behavior, but hard-coding what a "trial" is into the network object makes it very difficult for end-users to change the definition of a "trial" later.

**Humans** like to think about the following things:

* Trials
* Epochs
* Clamping
* ....

And parameters for the above. They particularly *don't* like to think about setup/teardown (who wants to remember to end plus phases before beginning minus phases?)

**Networks** don't care about human stuff. They care about commands like the following:

* Begin plus phase
* Cycle! Cycle! Cycle!
* End plus phase
* Begin minus phase
* ...

**Mixing** the two domains is *bad* because it forces humans and networks to know/care about stuff they dont' care about.

Solution? Treat the network like a computer processor. Take high-level programs that the user writes, compile them down into "network code", and let the network run the "network code."

### Benefits of this approach

* Separation of concerns
* Users can specify network behavior in a *declarative* way (good for reproducibility) without knowing about the internals of how the network, runs. This, I think, was a difficulty with Emergent.
* Well-researched foundations in compiler theory
* Enables parallelism in future due to async network event handling
* Potential for domain specific language if it becomes absolutely necessary

### Implementation tour

A **program** is just a **directed, rooted tree.** (see program above.)

To "compile" the program, we simply do a left-right depth-first-search preorder traversal of the tree.

*In case of compiler nerd people: this tree happens to be exactly the same as an abstract syntax tree (AST) :) *

In [ ]:
list(program.preorder())

Then, we filter it for "atomic events", or "network code" (i.e. things the network understands). Note that the Loop node and the Program node disappear in this case:

In [ ]:
list(program.atomic_stream())

Then, the network consumes each atomic event and either handles it (in the case of a cycle) or broadcasts it to every object in the network (BEGIN_PLUS_PHASE, HARD_CLAMP, etc.)

## Next steps

* Now that we have plus and minus phases up and running, we are veeeery close to XCAL!!!